In [1]:
import re

import numpy as np
import pandas as pd

In [2]:
players = pd.read_csv('../data/data.csv', encoding='utf-8')
league_team = pd.read_csv('../data/league_team_mapping.csv', encoding='utf-8')
league_rev = pd.read_csv('../data/league_country_level_revenue_mapping.csv', encoding='utf-8')

In [3]:
league = pd.merge(league_team, league_rev, on='League', how='left')
league.columns = [f'League {col}' for col in league.columns]

In [4]:
players = pd.merge(players, league, left_on='Club', right_on='League Team', how='left')

In [5]:
drop_cols = ['Unnamed: 0', 'ID', 'Name', 'Photo', 'Flag', 'Club Logo', 'Jersey Number', 'Joined', 'League Team', 'League League', 'Body Type']
players = players.drop(drop_cols, axis=1, errors='ignore')

In [6]:
def transform_money(df, colnames):
    df_copy = df.copy()
    for col in colnames:
        df_copy[col] = df_copy[col].apply(parse_money)
    return df_copy

def parse_money(x):
    if pd.isnull(x):
        return x
    m = re.search(r'€(\d+\.?\d*)(\w?)', x)
    value = float(m.group(1))
    unit = m.group(2)
    if unit == 'K':
        return value / 1000
    if unit == 'M':
        return value 
    return value / 1000000

In [7]:
def del_no_value(df):
    df_copy = df.copy()
    return df.loc[df.Value > 0]

In [8]:
def transform_height(df, colnames):
    df_copy = df.copy()
    for col in colnames:
        df_copy[col] = df_copy[col].apply(parse_height)
    return df_copy

def parse_height(x):
    if pd.isnull(x):
        return x
    m = re.search(r'(\d+)\'(\d+)', x)
    return int(m.group(1))*12 + int(m.group(2))

In [9]:
def transform_weight(df, colnames):
    df_copy = df.copy()
    for col in colnames:
        df_copy[col] = df_copy[col].apply(parse_weight)
    return df_copy

def parse_weight(x):
    if pd.isnull(x):
        return x
    m = re.search(r'(\d+)lbs', x)
    return int(m.group(1))

In [10]:
def transform_pos_ratings(df, colnames):
    df_copy = df.copy()
    for col in colnames:
        df_copy[col] = df_copy[col].apply(parse_pos_ratings)
    return df_copy

def parse_pos_ratings(x):
    if pd.isnull(x):
        return x
    m = re.search(r'(\d+)\+\d', x)
    return int(m.group(1))

In [11]:
def transform_loaned(df):
    df_copy = df.copy()
    loaned_players_idx = ~df_copy['Loaned From'].isna()
    df_copy.loc[loaned_players_idx, 'Club'] = df_copy.loc[loaned_players_idx, 'Loaned From']
    df_copy.loc[loaned_players_idx, 'Contract Valid Until'] = df_copy.loc[loaned_players_idx, 'Contract Valid Until'].str.slice(-4)
    df_copy['Loaned Out'] = False
    df_copy.loc[loaned_players_idx, 'Loaned Out'] = True
    df_copy = df_copy.drop('Loaned From', axis=1)
    return df_copy

In [12]:
def transform_work_rate(df):
    df_copy = df.copy()
    df_copy['Work Rate'] = df_copy['Work Rate'].str.split('/')
    df_copy['Off Work Rate'] = df_copy['Work Rate'].str[0]
    df_copy['Def Work Rate'] = df_copy['Work Rate'].str[1]
    df_copy = df_copy.drop('Work Rate', axis=1)
    return df_copy

In [13]:
def transform_league_revenue(df):
    df_copy = df.copy()
    df_copy['League Revenue'] = df_copy['League Revenue'].str.replace(',', '').astype(float)
    return df_copy

In [14]:
players = transform_money(players, ['Value', 'Release Clause', 'Wage'])
players = transform_league_revenue(players)
players = del_no_value(players)
players = transform_weight(players, ['Weight'])
players = transform_height(players, ['Height'])
players = transform_pos_ratings(players, 'LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB'.split(','))
players = transform_loaned(players)
players = transform_work_rate(players)

In [15]:
players.columns = [col.replace(' ', '_') for col in players.columns]

In [16]:
players.head()

,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,Preferred_Foot,International_Reputation,...,GKPositioning,GKReflexes,Release_Clause,League_Country,League_Level_Pyramid,League_Revenue,League_Revenue_Per_Team,Loaned_Out,Off_Work_Rate,Def_Work_Rate
0,31,Argentina,94,94,FC Barcelona,110.5,0.565,2202,Left,5.0,...,14.0,8.0,226.5,Spain,1.0,4479.0,223.95,False,Medium,Medium
1,33,Portugal,94,94,Juventus,77.0,0.405,2228,Right,5.0,...,14.0,11.0,127.1,Italy,1.0,2163.0,108.20,False,High,Low
2,26,Brazil,92,93,Paris Saint-Germain,118.5,0.290,2143,Right,5.0,...,15.0,11.0,228.1,France,1.0,1692.0,84.60,False,High,Medium
3,27,Spain,91,93,Manchester United,72.0,0.260,1471,Right,4.0,...,88.0,94.0,138.6,England,1.0,6562.0,328.10,False,Medium,Medium
4,27,Belgium,91,92,Manchester City,102.0,0.355,2281,Right,4.0,...,10.0,13.0,196.4,England,1.0,6562.0,328.10,False,High,High


In [17]:
players.to_csv('../data/data_cleaned.csv', index=False)